# Recuperación de pacientes de XiO
-------------------
## Descripción
Mediante este cuaderno datos de *pacientes archivados en XiO se convierten a formato DICOM* para su importación en ARIA.

Los datos archivados en XiO están almacenados en un servidor del Hospital al que se accede mediante `ftp`

La conversión utiliza los programas `plastimatch`y `Velocity`

`plastimatch` recupera de los datos de XiO las imágenes de CT, las estructuras y la dosis.

`Velocity` añade un plan *contenedor* para poder importar la dosis en ARIA.

## Procedimiento

* Recuperar del servidor del Hospital el fichero con los datos archivados del paciente
* Desarchivar los datos de XiO
* Correr los scripts de recuperación de las imágenes CT, las estructuras y la dosis
* Fijar un marco de referencia común para las imágenes CT, las estructuras y la dosis
* Crear en Velocity un plan contenedor para la dosis
* Importar en ARIA
* Limpiar
  
### Recuperación de los datos archivados

Esta fase requiere del uso de un cliente `ftp`.

Se puede utilizar el siguiente código o si se prefiere un cliente gráfico, se recomienda instalar `FileZilla`.

**La siguiente celda se tiene que modificar** para introducir el número de historia del paciente.

In [ ]:
PatientID = '613964'

En XiO los pacientes con diferentes cursos de tratamiento se archivaron cada vez que un curso se acababa.

El archivado se realizó de forma integral de manera que cada archivo contiene a los anteriores.

En la carpeta del paciente se tiene que elegir el archivado con la fecha más reciente.

La siguiente celda muestra el contenido de la carpeta del paciente en el servidor `ftp`

In [ ]:
import ftplib

# Definir el servidor ftp y conectar
ftp = ftplib.FTP('10.137.2.113')
print(ftp.login("radioterapia","radioterapia"))
from pathlib import Path

# Cambiar el directorio en el servidor al directorio del paciente
ftp.cwd(str(Path('/', 'Archivo', 'NoCategorizado', PatientID)))

# Listar los datos archivados del paciente
ArchivedFiles = ftp.nlst()
print('Archivos del paciente:')
import pandas as pd
pd.DataFrame(ArchivedFiles, columns = ['Archivo'])

**La siguiente celda se tiene que modificar** para actualizar el índice del archivo del archivo que se quiere recuperar del servidor ftp

In [ ]:
ArchivedFileIndex = 0
PatientArchivedFile = ArchivedFiles[ArchivedFileIndex]

Hacer una copia del archivo del paciente en `XiOAriaDrive`

In [ ]:
localfile = str(Path('/', 'home', 'radiofisica', 'Shares', 'Radiofisica','XiOAriaDrive', PatientArchivedFile))
with open(localfile,'wb') as localf:
    ftp.retrbinary("RETR " + PatientArchivedFile, localf.write)

Cerrar la conexión con el servidor

In [ ]:
ftp.quit()

Si se ha optado por usar `FileZila` los datos de conexión son:
* Servidor: **10.137.2.113**
* Nombre de usuario: **radioterapia**
* Contraseña: **radioterapia**

Pulsar sobre el botón `Conexión rápida`

En Sitio remoto navegar en la ruta `Archivo/No Categorizado` para seleccionar la carpeta del paciente que se quiere recuperar

Seleccionar el archivo`.tgz`. Si hay varios elegir el más reciente

Arrastrar el archivo sobre la carpeta `Radiofisica/XiOAriaDrive` del Sitio Local

![FileZillaMainScreen](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/FileZillaMainScreen.PNG)
### Desarchivar los datos de XiO

Definir el directorio de trabajo base

In [ ]:
# Referenciar
baseworkdir = '/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO'

from pathlib import Path
if not Path(baseworkdir).exists():
    !mkdir -p $baseworkdir

**La siguiente celda se tiene que modificar** para definir el nombre del archivo con los datos archivados en XiO

In [ ]:
if 'PatientArchivedFile' not in locals():
    PatientArchivedFile = '584955_SANCHEZ_CABALLERO_LUCIA_Mar_14_2011.tgz'

Desarchivar

In [ ]:
%%capture
!tar -xvzf $PatientArchivedFile -C $baseworkdir

Mostrar la estructura de carpetas de los datos desarchivados

In [ ]:
!tree -d $baseworkdir/patient

Recuperar datos relevantes.

Número de historia y nombre y apellidos del paciente

In [ ]:
# Número de historia
if 'PatientID' not in locals():
    PatientID = PatientArchivedFile.split('_')[0]

# Nombre y apellidos
demographicfile = !find $baseworkdir/patient/$PatientID -name demographic -print
with open(demographicfile[0], 'r', encoding='latin') as demogf:
  demographicdata = demogf.read()

ApellidosNombre = demographicdata.split('\n')[2]

print(f'Historia: {PatientID}\nApellidos y nombre: {ApellidosNombre}')

**La siguiente celda se tiene que modificar** para definir el estudio CT y el plan que se quiere recuperar

In [ ]:
CT = 'CT1'
Plan ='ptv1'

### Scripts de recuperación

Importación de módulos

In [ ]:
# - Manipulación de archivos DICOM
import pydicom as dicom
# - Conversión del formato nativo de XiO a DICOM
import pyplastimatch as plast
# - Rutas y comprobaciones de existencia de ficheros
from os import path, getcwd
from glob import glob
from sys import path as syspath

Crear la carpeta `scripts` en el directorio de trabajo base y poblarla con los scripts que complementan el funcionamiento de `plastimatch`

In [ ]:
# Descargar los scripts de corrección
if Path(baseworkdir + '/scripts').exists():
  !rm -rf $baseworkdir/scripts
!mkdir -p $baseworkdir/scripts
repodir = '/home/radiofisica/Shares/Radiofisica/csr/XiO/XiOrecovery/scripts'
!cp  $repodir/xioss $baseworkdir/scripts 
!cp  $repodir/genindex $baseworkdir/scripts 
!cp  $repodir/deltact $baseworkdir/scripts 
!cp  $repodir/dcmcoher.py $baseworkdir/scripts 

# Añadir al sistema la ruta de los scripts
if baseworkdir + '/scripts' not in syspath:
  syspath.insert(0, baseworkdir + '/scripts')

Recuperación de las estructuras

In [ ]:
# Si no existe crear el índice de imágenes CT y las máscaras `.mha` de todas las estructuras
if not Path(baseworkdir + '/patient/' + PatientID + '/anatomy/studyset/' + CT + '/index.dat').exists():
    # Copiar el script para la creación del índice de cortes de CT
    !cp $baseworkdir/scripts/genindex $baseworkdir/patient/$PatientID/anatomy/studyset/$CT
    
    # Adjudicar permisos de ejecución y crear el índice
    !chmod +x $baseworkdir/patient/$PatientID/anatomy/studyset/$CT/genindex
    cwd = getcwd()
    %cd -q $baseworkdir/patient/$PatientID/anatomy/studyset/$CT
    print('Generar el índice de imáagenes de CT...')
    !./genindex
    %cd -q $cwd

# Crear la carpeta para los archivos máscara con las estructuras
!mkdir -p $baseworkdir/patient/$PatientID/anatomy/studyset/$CT/ss

# Copiar el script para la creación de las estructuras
!cp $baseworkdir/scripts/xioss $baseworkdir/patient/$PatientID/anatomy/studyset/$CT

# Correr el script de recuperación de las estructuras
!chmod +x $baseworkdir/patient/$PatientID/anatomy/studyset/$CT/xioss
print('Recuperar estructuras...')
!$baseworkdir/patient/$PatientID/anatomy/studyset/$CT/xioss --input $baseworkdir/patient/$PatientID --plan $Plan --studyset $CT > xioss.log

Conversión del CT, las estructuras y la dosis desde formato XIO a formato DICOM 

**La siguiente celda se tiene que modificar** para indicar si el paciente se simuló en prono y hay incoherencia entre la dosis y el CT.

En XiO planificábamos los pacientes en prono sin cambiar la indicación de orientación del paciente en el CT.

En algunos pacientes se indicó en XiO la orientación real del paciente Treatment orientation: Head Patient position on couch: Prone

Esto produce una inconsitencia en el estudio exportado de XiO cuando se importa en ARIA: las estructuras y la dosis aparecen correctamente orientadas pero las imágenes de CT aparecen volteadas.

Para solucionarlo en la siguiente celda moddificar `prono_correction = True` 

In [ ]:
# Variables de pyplastimatch
prono_correction = False
verbose = True
path_to_logfile_xiodcm = './' + PatientID + '_xiodcm.log'

# Opciones de plastimatch convert
if prono_correction:
    convert_args_rt ={'input' :  baseworkdir + '/patient/' + PatientID  + '/anatomy/studyset/' + CT + '/DCMData',
                      'input-prefix' : baseworkdir + '/patient/' + PatientID  + '/anatomy/studyset/' + CT + '/ss',
                      'input-dose-xio' : baseworkdir + '/patient/' + PatientID  + '/plan/' + Plan + '/dose.1',
                      'output-dicom' : baseworkdir + '/../' + PatientID + '/plastimatch', 'patient-pos' : 'hfs'}
else:
    convert_args_rt ={'input' :  baseworkdir + '/patient/' + PatientID  + '/anatomy/studyset/' + CT + '/DCMData',
                      'input-prefix' : baseworkdir + '/patient/' + PatientID  + '/anatomy/studyset/' + CT + '/ss',
                      'input-dose-xio' : baseworkdir + '/patient/' + PatientID  + '/plan/' + Plan + '/dose.1',
                      'output-dicom' : baseworkdir + '/../' + PatientID + '/plastimatch'}

# Correr el script de conversión de los datos XiO en estudio DICOM
plast.convert(verbose=verbose, path_to_log_file=path_to_logfile_xiodcm, **convert_args_rt )

# Crear una carpeta para exportar los archivos trasformados en Velocity AI
velocitydir = baseworkdir + '/../' + PatientID + '/velocity'
!mkdir $velocitydir

### Corrección de los marcos de referencia

In [ ]:
# Referencia al directorio de trabajo actual
cwd = getcwd()
# Cambiar al directorio patient
%cd -q $baseworkdir/patient

# Copiar el script para determinar el delta entre los marcos de referencia del estudio CT y el plan
!cp $baseworkdir/scripts/deltact .

## Asegurar que el archivo se actualiza
if Path('./deltaframes').exists():
  !rm -f ./deltaframes
## Adjudicar permisos de ejecución y calcular el delta
!chmod +x deltact
!./deltact --input $PatientID --studyset $CT > deltact.log

## Regresar al directorio base
%cd -q $baseworkdir

# Importar las herramientas de corrección
from dcmcoher import correctImagePositionPatientInCTImages

# Establecer el mismo marco de referencia en todo el estudio
print('Establecer el mismo marco de referencia para las imágenes CT, las estructuras y la dosis...')
correctImagePositionPatientInCTImages(patientID=PatientID, studyset=CT, dcmdir=baseworkdir + '/../' + PatientID + '/plastimatch')

# Volver al directorio inicial
%cd -q $cwd

### Crear plan contenedor (Velocity)

Entrar en `Velocity AI`.

Las credenciales del usuario genérico son:

* Nombre: **velocity**
* Contraseña: **Velocity01**

![VelocityLogin](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityLogin.png) )

En la pantalla principal pulsar sobre el icono de Menú ![VelocityMenu](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityMenu.png) y la opción de Importar ![VelocityImport](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityImport.png)

Navegar hasta la carpeta que contiene los archivos generados mediante `plastimatch`, ruta: *Radiofisica/XiOAriaDrive/**Historia**/plastimatch*. Pulsar en ![VelocitySeleccionarCarpeta](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocitySeleccionarCarpeta.png)

`Velocity`leerá el contenido de la carpeta y deberá identificar un estudio de CT, un conjunto de estructuras y un archivo de dosis. Pulsar sobre `Import`

![VelocityImportForm](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityImportForm.png)

Tras la importación `Velocity` muestra un log de importación. Cerrar la ventana pulsando sobre ![VelocityCerrar](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityCerrar.png)

![VelocityImportLog](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityImportLog.png)

Hacer doble click sobre el paciente importado

![VelocitySelectPatient](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocitySelectPatient.png)

En la siguiente ventana `Velocity`muestra los volúmenes importados (CT, estructuras y dosis) y las relaciones entre ellos

![VelocityVolumes](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityVolumes.png)

Seleccionar el botón `Start Navigator` ![VelocityStartNavigator](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityStartNavigator.png) y en el menú `Plan Operations` la opción `Create Container Plan`

`Velocity` muestra un resumen de las acciones que va a realizar, de los requisitos necesarios, de los objetos que se generarán y del procedimiento.

En nuestro caso elegiremos no exportar, y como volúmenes el estudio de CT, las estructuras y las dosis importadas de plastimatch.

Pulsar el botón `Start` para iniciar el procedimiento

![VelocityCreateContainerPlan](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityCreateContainerPlan.png)

Elegir no enviar y continuar

![VelocityDICOMLocation](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityDICOMLocation.png)

Seleccionar el volumen de dosis y continuar

![VelocityDoseVolume](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityDoseVolume.png)

Seleccionar el estudio de CT y continuar

![VelocityCTStudysetVolume](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityCTStudysetVolume.png)

Seleccionar el conjunto de estructuras y continuar

![VelocityStructureSetVolume](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityStructureSetVolume.png)

Dar nombre al plan contenedor y continuar

![VelocityPlanName](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityPlanName.png)

Finalizar el procedimiento 

![VelocityCreateContainerPlanFinish](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityCreateContainerPlanFinish.png)

`Velocity` muestra un resumen del proceso realizado. Pulsar el botón `Finished`

![VelocityCreateContainerPlanSumary](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityCreateContainerPlanSumary.png)

En el menú `Volumes` seleccionar el volumen de dosis **procesada** y con el botón derecho seleccionar la opción `Export`

![VelocityDoseExport](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityDoseExport.png)

Seleccionar la carpeta *velocity* dentro de la carpeta con el número de historia del paciente. Pulsar el botón `Seleccionar carpeta`

![VelocityExportFolder](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityExportFolder.png)

`Velocity` nos da un aviso sobre *aliasing*. Aceptamos

![VelocityAliasingWarning](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityAliasingWarning.png)

Repetimos el paso anterior para el estudio de CT

![VelocityCTExport](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityCTExport.png)

Para el conjunto de estructuras

![VelocityStructureSetExport](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityStructureSetExport.png)

Y para el plan contenedor

![VelocityPlanExport](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/VelocityPlanExport.png)

### Importar en ARIA

Para facilitar la importación en ARIA comenzar copiando o moviendo la carpeta *velocity* a la unidad local **D:** del ordenador en el que estemos trabajando.

En Aria ir a `DICOM > Import Export`

En el paso *Filter Selecction* seleccionar el filtro de importación DICOM

![AriaImportImages](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/AriaImportImages.png)

En el paso *Import Selection* seleccionar la carpeta *velocity* de la unidad local **D:**

![AriaImportSelectFolder](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/AriaImportSelectFolderRed.png)

En el segundo paso de *Import Selection* confirmar que se han leído el estudio de CT, el conjunto de estructuras, el plan contenedor y la dosis, y que todos están seleccionados

![AriaImportSelection](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/AriaImportSelection2.png)

En el paso *Patient Selection* confirmar que los archivos a importar corresponden al paciente seleccionado

![AriaPatientSelection](https://raw.githubusercontent.com/csarux/XiOrecovery/main/jupyterlab/Images/AriaPatientSelection.png)

Para terminar la importación completar los pasos *Connection* y *Sumary*

Por último abrir el estudio en Eclipse para comprobar que todo se ha realizado correctamente.

#### Renormalización de la dosis

Eclipse y XiO tratan de manera diferente la normalización de la dosis.

Si aceptamos por defecto la normalización en Eclipse de las dosis calculadas en XiO, Eclipse asignará el 100% a la dosis máxima del plan $D_{max}$.

Eclipse considera que el plan contenedor se realiza en una única sesión.

Para asignar unas curvas de isodosis tales que en dosis relativa el 100% corresponda a la dosis de prescripción, en Eclipse en la pestaña `Dose` tenemos que:

* Poner en la casilla `Total Dose [Gy]` el valor de la dosis total de prescripción del tratamiento $D_T$.
* En `Plan Normalization Mode` ajustar `Plan Normalization Value` según la siguiente expresión
  \begin{equation}
      \text{Plan Normalization Value} = \frac{D_T}{D_{max}}100.
  \end{equation}

**Nota**: La dosis total de prescripción $D_T$ se puede consultar en **Selene > Departamentales** en los pdf con los informes de tratamiento.

### Limpiar los archivos intermedios

In [ ]:
# Borrar las carpetas colocadas en XiOAriaDrive
!rm -rf $baseworkdir
patientdir = '/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/' + PatientID
!rm -rf $patientdir

# Borrar el archivo traído mediante ftp con los datos archivados del paciente
from os import remove
PatientArchivedFileAbsolutePath = '/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/' + PatientArchivedFile
remove(PatientArchivedFileAbsolutePath)

# Borrar el log de recuperación de estructuras
remove('/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/xioss.log')

# Borrar el log de plastimatch
remove('/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/' + PatientID + '_xiodcm.log')

# Borrar si existe el archivo de bloqueo de Focus/XiO
from os.path import exists
if exists('/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/archive_configuration'):
    remove('/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/archive_configuration')

Borrar manualmente la carpeta *velocity* de la unidad local **D:**